## Wk7 Assignment

## Lincoln Brown

## DSC550-T302

## Professor Werner

## Part 1: PCA and Variance Threshold in a Linear Regression

In [309]:
import numpy as np
import os
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

## Step 1: Import the Data
Import the housing data as a dataframe and ensure that the data is loaded properly.

In [310]:
if not os.getcwd().__contains__('house-prices-advanced'):
    os.chdir('house-prices-advanced-regression-techniques')
os.listdir()

['test.csv', 'data_description.txt', 'train.csv', 'sample_submission.csv']

In [311]:
df = pd.read_csv('train.csv')
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


## Step 2: Drop Columns

Drop the "Id" column and any features that are missing more than 40% of their values.

In [312]:
cleaned_df = df.copy()
print(f"There are {cleaned_df.shape[0]} records and {cleaned_df.shape[1]} features in the dataset.")
if "Id" in cleaned_df.columns:
    cleaned_df = cleaned_df.drop("Id", axis=1)
cleaned_df.isna().sum()

There are 1460 records and 81 features in the dataset.


MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
Street             0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 80, dtype: int64

In [313]:
# Drop any features that are missing more than 40% of their values
print("Dropping any features with more than 40% of their values missing.")
total_length = df.shape[0]
for col in cleaned_df.columns:
    col_nulls = cleaned_df.loc[:,col].isna().sum()
    percent_null = col_nulls / total_length
    if percent_null > .4:
        print(f"{col} # of NaNs: {col_nulls}.  {percent_null:.2%}")
        print(f"Dropping {col}")
        cleaned_df = cleaned_df.drop(col, axis=1)

Dropping any features with more than 40% of their values missing.
Alley # of NaNs: 1369.  93.77%
Dropping Alley
MasVnrType # of NaNs: 872.  59.73%
Dropping MasVnrType
FireplaceQu # of NaNs: 690.  47.26%
Dropping FireplaceQu
PoolQC # of NaNs: 1453.  99.52%
Dropping PoolQC
Fence # of NaNs: 1179.  80.75%
Dropping Fence
MiscFeature # of NaNs: 1406.  96.30%
Dropping MiscFeature


In [314]:
print(f"After cleaning, there are {cleaned_df.shape[0]} records and {cleaned_df.shape[1]} features.")

After cleaning, there are 1460 records and 74 features.


## Step 3: Fill in Missing Numeric Values

For numerical columns, fill in any missing data with the median value.

In [315]:
cleaned_df.dtypes.value_counts()

object     37
int64      34
float64     3
Name: count, dtype: int64

In [316]:
cleaned_df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2010,WD,Normal,210000
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,112,0,0,0,0,4,2010,WD,Normal,142125


In [149]:
# To impute the NaNs in the numeric columns, we need to first select those columns
numeric_cols = cleaned_df.select_dtypes(include=['number']).columns
print(f"There are {len(numeric_cols)} numeric columns")

# Get the median for each numeric column
medians = cleaned_df.loc[:,numeric_cols].median() 
# Get a dataframe mask of true false values for NaNs. NaNs will be True, Non-missing values will be False
numeric_mask = cleaned_df.loc[:,numeric_cols].isna()

# pandas dataframe.where statements replace values marked as false, with corresponding value from other.
# Therefore, we need to bitwise NOT the mask, making the NaNs marked as True, False thus replacing them with medians
cleaned_df.loc[:,numeric_cols] = cleaned_df.loc[:,numeric_cols].where(cond=~numeric_mask, other=medians, axis=1)


There are 37 numeric columns


In [150]:
# Print the columns with NaNs
nans = cleaned_df.isna().sum()
nans[nans > 0]

BsmtQual        37
BsmtCond        37
BsmtExposure    38
BsmtFinType1    37
BsmtFinType2    38
Electrical       1
GarageType      81
GarageFinish    81
GarageQual      81
GarageCond      81
dtype: int64

## Fill in Missing Categorical Values

For categorical columns, fill in any missing data with the most common value (mode).

In [151]:
# Now we will apply the same principle to the categorical columns
# First we isolate the categoric columns
categoric_cols = cleaned_df.select_dtypes(include=['object']).columns
print(f"There are {len(categoric_cols)} categorical columns")
# Get the mode for each categorical column
modes = cleaned_df.loc[:,categoric_cols].mode()
# Create a mask for the categoric columns marking NaNs as True and non-missing values as False
categoric_mask = cleaned_df.loc[:, categoric_cols].isna()

# Use a negated where statement to apply the modes to the categorical NaN values
cleaned_df.loc[:, categoric_cols] = cleaned_df.loc[:,categoric_cols].where(cond=~categoric_mask, other=modes, axis=1)

There are 37 categorical columns


In [152]:
# Confirm there are no more columns with NaNs
nans = cleaned_df.isna().sum()
nans[nans > 0]

Series([], dtype: int64)

## Convert to Dummy Variables

Convert the categorical columns to dummy variables.

In [153]:
dummy_df = pd.get_dummies(data=cleaned_df)

MSSubClass               False
LotFrontage              False
LotArea                  False
OverallQual              False
OverallCond              False
                         ...  
SaleCondition_AdjLand    False
SaleCondition_Alloca     False
SaleCondition_Family     False
SaleCondition_Normal     False
SaleCondition_Partial    False
Length: 267, dtype: bool

## Step 6: Split the data

Split the data into a training and test set, where the SalePrice column is the target.

MSSubClass               False
LotFrontage              False
LotArea                  False
OverallQual              False
OverallCond              False
                         ...  
SaleCondition_AdjLand    False
SaleCondition_Alloca     False
SaleCondition_Family     False
SaleCondition_Normal     False
SaleCondition_Partial    False
Length: 267, dtype: bool

In [157]:
# Split features and target variable

# Get features
X = dummy_df.drop('SalePrice', axis=1) 

# Get Target Variable
y = dummy_df['SalePrice'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)


## Step 7: Run a Linear Regression

Run a linear regression and report the R2-value and RMSE on the test set.

In [282]:
%time
# Create a Logistic Regression object
linear_regression = LinearRegression()
scaled_linear_regression = LinearRegression()
minmax_linear_regression = LinearRegression()

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 7.15 µs


In [283]:
%time
# Train the linear regression model
linear_reg_model = linear_regression.fit(X_train, y_train)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 10 µs


In [290]:
# Use Standard Scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [285]:
%time
# Train the scaled linear regression model
scaled_linear_reg_model = scaled_linear_regression.fit(X_train_scaled, y_train)

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 14.1 µs


In [286]:
# Predict on the non-standardized model
y_pred = linear_reg_model.predict(X_test)

# Print the R2 and RMSE
r2_val = r2_score(y_test, y_pred)
rmse_val = mean_squared_error(y_test, y_pred, squared=False) # Set squared to false to get RMSE

print(f"The Linear Regression Model achieved an R2 Score of {r2_val:.2f} and a Root Mean Squared Error of {rmse_val:,.2f}.")

The Linear Regression Model achieved an R2 Score of 0.88 and a Root Mean Squared Error of 27,917.20.


In [287]:
# Predict on the standardized model
y_pred_scaled = scaled_linear_regression.predict(X_test)

# Print the R2 and RMSE
scaled_r2_val = r2_score(y_test, y_pred_scaled)
scaled_rmse_val = mean_squared_error(y_test, y_pred_scaled, squared=False) # Set squared to false to get RMSE

print(f"The Standardized Linear Regression model achieved an R2 score of {scaled_r2_val:.2f} and a Root Mean Squared Error of {scaled_rmse_val:,.2f}.")

The Standardized Linear Regression model achieved an R2 score of -184828930721311147322293354496.00 and a Root Mean Squared Error of 34,385,264,019,420,864,512.00.


/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


In [298]:
# Use Min Max Scaler
min_max_scaler = MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_test_minmax = min_max_scaler.transform(X_test)

In [299]:
%time
# Train the mix max linear regression model
minmax_linear_reg_model = minmax_linear_regression.fit(X_train_minmax, y_train)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 11 µs


In [275]:
# Predict on the MinMax model
y_pred_minmax = minmax_linear_regression.predict(X_test_minmax)

# Print the R2 and RMSE
minmax_r2_val = r2_score(y_test, y_pred_minmax)
minmax_rmse_val = mean_squared_error(y_test, y_pred_minmax, squared=False) # Set squared to false to get RMSE

print(f"The MinMax Standardized Linear Regression model achieved an R2 score of {minmax_r2_val:.2f} and a Root Mean Squared Error of {minmax_rmse_val:,.2f}.")

The MinMax Standardized Linear Regression model achieved an R2 score of -49677949957227970560.00 and a Root Mean Squared Error of 563,727,499,168,690.62.


In [300]:
%time

# Create a Linear Regression model
linear_regression = LinearRegression()

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 13.8 µs


In [301]:
%time
# Train the linear regression model
linear_reg_model = linear_regression.fit(X_train, y_train)
    
# Predict on the non-standardized model
y_pred = linear_reg_model.predict(X_test)

# Print the R2 and RMSE
r2_val = r2_score(y_test, y_pred)
rmse_val = mean_squared_error(y_test, y_pred, squared=False) # Set squared to false to get RMSE

print(f"The Linear Regression Model achieved an R2 Score of {r2_val:.2f} and a Root Mean Squared Error of {rmse_val:,.2f}.")

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 8.82 µs
The Linear Regression Model achieved an R2 Score of 0.88 and a Root Mean Squared Error of 27,917.20.


In [302]:
%time
# Use Standard Scaler
# Use Standard Scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the scaled linear regression model
%time scaled_linear_reg_model = linear_regression.fit(X_train_scaled, y_train)

# Predict on the standardized model
y_pred_scaled = scaled_linear_reg_model.predict(X_test_scaled)

# Print the R2 and RMSE
scaled_r2_val = r2_score(y_test, y_pred_scaled)
scaled_rmse_val = mean_squared_error(y_test, y_pred_scaled, squared=False)

print(f"The Standardized Linear Regression model achieved an R2 score of {scaled_r2_val:.2f} and a Root Mean Squared Error of {scaled_rmse_val:,.2f}.")

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 12.9 µs
CPU times: user 299 ms, sys: 112 ms, total: 411 ms
Wall time: 120 ms
The Standardized Linear Regression model achieved an R2 score of -85113805034582381166592.00 and a Root Mean Squared Error of 23,333,903,147,916,024.00.


## Step 8: Create a PCA

Fit and transform the training features with a PCA so that 90% of the variance is retained (see section 9.1 in the Machine Learning with Python Cookbook).

In [304]:
# Create a PCA that will retain 90% of variance
pca = PCA(n_components=.9, whiten=True)

In [305]:
# Fit and transform the training features
X_train_pca = pca.fit_transform(X_train)

In [306]:
X_train_scaled_pca = pca.fit_transform(X_train_scaled)
X_train_scaled_pca.shape

(1168, 136)

In [307]:
X_train_minmax_pca = pca.fit_transform(X_train_minmax)
X_train_minmax_pca.shape

(1168, 70)

## Step 9: Number of Features in PCA matrix

How many features are in the PCA-transformed matrix?


In [308]:
# Display the results
print(f"Original number of features: {X_train.shape[1]}")
print(f"Number of features after PCA reduction: {X_train_pca.shape[1]}")
print(f"Number of scaled features after PCA reduction {X_train_scaled_pca.shape[1]}")
print(f"Number of minmax features after PCA reduction {X_train_minmax_pca.shape[1]}")

Original number of features: 266
Number of features after PCA reduction: 1
Number of scaled features after PCA reduction 136
Number of minmax features after PCA reduction 70


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
1216,90,68.0,8930,6,5,1978,1978,0.0,0,0,...,False,False,False,True,False,False,False,False,True,False
339,20,66.0,12400,6,7,1958,1998,176.0,585,0,...,False,False,False,True,False,False,False,False,True,False
1057,60,69.0,29959,7,6,1994,1994,0.0,595,0,...,False,False,False,True,False,False,False,False,True,False
482,70,50.0,2500,7,8,1915,2005,0.0,299,0,...,False,False,False,True,False,False,False,False,True,False
529,20,69.0,32668,6,3,1957,1975,0.0,1219,0,...,False,False,False,True,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1393,190,60.0,10800,6,7,1905,2000,0.0,0,0,...,False,False,False,True,False,False,False,False,True,False
1344,60,85.0,11103,7,5,2006,2006,0.0,0,0,...,False,True,False,False,False,False,False,False,False,True
527,60,67.0,14948,9,5,2008,2008,268.0,1330,0,...,False,True,False,False,False,False,False,False,False,True
1149,70,50.0,9000,7,9,1920,1988,0.0,624,0,...,False,False,False,True,False,False,False,False,True,False


In [280]:
pca = PCA(n_components=0.90)
scaler = StandardScaler()
features = scaler.fit_transform(X_train)
pca_X_train = pca.fit_transform(features)
pca_X_train.shape

(1168, 136)